----Reconstruction of pairwise community model-----

In [ ]:
##1Reconstruction of pairwise community model- GCM_P1_P2##
import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/ksp_P2.xml') #Uploading Individual GEM ksp_P2#
model2 = cobra.io.read_sbml_model(r'Final_GEM/ml_P1.xml') #Uploading Individual GEM ml_P1#

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

#optimizing the model1#
solution=model1.optimize()
solution.objective_value
Vbiop2 = model1.reactions.bio1.flux ## Biomass in individual GEM p2
Vbiop2 = round(Vbiop2, 3)


#identifying the blocked reactions#
cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

#optimizing the model2#
solution=model2.optimize()
solution.objective_value
Vbiop1 = model2.reactions.bio1.flux ## Biomass in individual GEM p1
Vbiop1 = round(Vbiop1, 3)

#identifying the blocked reactions#
cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

##Defining the species-specific metabolite annotation##
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ksp"
        i.id=j
        i.compartment="c0ksp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ml"
        i.id=j
        i.compartment="c0ml"

count=0

##Defining the species-specific reaction annotation##
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0

##Defining the species-specific biomass reaction annotation##
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0

##Defining the species-specific biomass reaction annotation##
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

##Defining Community model
from cobra import Model, Reaction, Metabolite
model = Model('community_model1')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )

##Defining the community biomass reactions based on the individual bacterial growth rate##
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ksp"): -0.43,
                          model2.metabolites.get_by_id("cpd11416_c0ml"): -0.57,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))


##defining Objective Function
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

##Writing the community model##
cobra.io.write_sbml_model(model,'Output_GCM/GCM_P1_P2_new.xml')##writing community model

##Running FBA##
F = model.optimize()
flux_p1_p2=F.fluxes
flux_p1_p2.to_csv("Output_FBA/FBA_p1_p2_new.csv")

##PGSI##
Vbiocomp1p2 = model.reactions.Bio1.flux
Vbiocomp2 = model.reactions.bio1ksp.flux ##biomass of P2 in community GEM
Vbiocomp2 = round(Vbiocomp2, 3)

Vbiocomp1 = model.reactions.bio1ml.flux ##biomass of P1 in community GEM
Vbiocomp1 = round(Vbiocomp1, 3)

PGSIp2p1 = ((Vbiocomp1-Vbiop1)/Vbiop1)
PGSIp2p1 = round(PGSIp2p1, 3)
PGSIp1p2 = ((Vbiocomp2-Vbiop2)/Vbiop2)
PGSIp1p2 = round(PGSIp1p2, 3)

print('PGSI_P1-->P2 :', PGSIp1p2)
print('PGSI_P2-->P1:', PGSIp2p1)

print('Avg PGSI: ', ((PGSIp1p2+PGSIp2p1)/2))

In [ ]:
##2Reconstruction of pairwise community model- GCM_P1_P3##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/ml_P1.xml') #Uploading Individual GEM#
model2 = cobra.io.read_sbml_model(r'Final_GEM/csp_P3.xml') #Uploading Individual GEM#

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

#optimising the model1#
solution=model1.optimize()
solution.objective_value
Vbiop1 = model1.reactions.bio1.flux ## Biomass in individual GEM p1
Vbiop1 = round(Vbiop1, 3)



cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

# optimizing the model2#
solution=model2.optimize()
solution.objective_value
Vbiop3 = model2.reactions.bio1.flux ## Biomass in individual GEM p3
Vbiop3 = round(Vbiop3, 3)


cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

##Defining the species-specific metabolite annotation##
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ml"
        i.id=j
        i.compartment="c0ml"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"csp"
        i.id=j
        i.compartment="c0csp"

##Defining the species-specific reaction annotation##
count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)
##Defining the species-specific biomass reaction annotation##

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

##Defining Community model

from cobra import Model, Reaction, Metabolite
model = Model('community_model2')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )

##Defining the community biomass reactions based on the individual bacterial growth rate##
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ml"): -0.69,
                          model2.metabolites.get_by_id("cpd11416_c0csp"): -0.31,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining community objective function##
model.objective = 'Bio1'
print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

##Writing the community model##
cobra.io.write_sbml_model(model,'Output_GCM/GCM_P1_P3.xml')

#Flux Balance Analysis#
F = model.optimize()
flux_p1_p3=F.fluxes
flux_p1_p3.to_csv("Output_FBA/FBA_p1_p3.csv")


##PGSI##
Vbiocomp1p3 = model.reactions.Bio1.flux
Vbiocomp1 = model.reactions.bio1ml.flux  ##biomass of P1 in community GEM
Vbiocomp1 = round(Vbiocomp1, 3)

Vbiocomp3 = model.reactions.bio1csp.flux  ##biomass of P3 in community GEM
Vbiocomp3 = round(Vbiocomp3, 3)

PGSIp3p1 = ((Vbiocomp1-Vbiop1)/Vbiop1)
PGSIp3p1 = round(PGSIp3p1, 3)

PGSIp1p3 = ((Vbiocomp3-Vbiop3)/Vbiop3)
PGSIp1p3 = round(PGSIp1p3, 3)


print('PGSI_P1-->P3:', PGSIp1p3)
print('PGSI_P3-->P1:', PGSIp3p1)

print('Avg PGSI: ', ((PGSIp1p3+PGSIp3p1)/2))

In [ ]:
##3Reconstruction of pairwise community model- GCM_P1_P4##

model1 = cobra.io.read_sbml_model(r'Final_GEM/ml_P1.xml') #Uploading Individual GEM p1#
model2 = cobra.io.read_sbml_model(r'Final_GEM/osp_P4.xml') #Uploading Individual GEM 4#

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

#optimising the model1#
solution=model1.optimize()
solution.objective_value
Vbiop1 = model1.reactions.bio1.flux ## Biomass in individual GEM p1
Vbiop1 = round(Vbiop1,3)

cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

#optimising the value#
solution=model2.optimize()
solution.objective_value
Vbiop4 = model2.reactions.bio1.flux ## Biomass in individual GEM p4
Vbiop4 = round(Vbiop4,3)

cobra.flux_analysis.find_blocked_reactions(model2)


##Defining the species-specific metabolite annotation##

for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ml"
        i.id=j
        i.compartment="c0oml"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"osp"
        i.id=j
        i.compartment="c0osp"
        
##Defining the species-specific reaction annotation##

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

## Species-specific Biomass reaction annotation##

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)


##defining communtiy model

from cobra import Model, Reaction, Metabolite
model = Model('community_model3')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )

##defining community biomass reactions

reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ml"): -0.73,
                          model2.metabolites.get_by_id("cpd11416_c0osp"): -0.27,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining community objective function

model.objective = 'Bio1'
print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

##Writing Community model

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P1_P4.xml')


##Running FBA
F = model.optimize()
flux_p1_p4=F.fluxes
flux_p1_p4.to_csv("Output_FBA/FBA_p1_p4.csv")

##Calculating PGSI
Vbiocomp1p4 = model.reactions.Bio1.flux
Vbiocomp1 = model.reactions.bio1ml.flux   ##Biomass of P1 in the community GEM
Vbiocomp1 = round(Vbiocomp1,3) 

Vbiocomp4 = model.reactions.bio1osp.flux   
Vbiocomp4 = round(Vbiocomp4, 3)

PGSIp4p1 = ((Vbiocomp1-Vbiop1)/Vbiop1)
PGSIp4p1 = round(PGSIp4p1,3)
PGSIp1p4 = ((Vbiocomp4-Vbiop4)/Vbiop4)
PGSIp1p4 = round(PGSIp1p4,3)

print('PGSI_P1-->P4:', PGSIp1p4)
print('PGSI_P4-->P1:', PGSIp4p1)
print('Avg PGSI: ', ((PGSIp1p4+PGSIp4p1)/2))

Reconstruction of pairwise community model- GCM_P1_P5

In [ ]:
##4Reconstruction of pairwise community model- GCM_P1_P5##


model1 = cobra.io.read_sbml_model(r'Final_GEM/bsp_P5.xml') #Uploading Individual GEM p5#
model2 = cobra.io.read_sbml_model(r'Final_GEM/ml_P1.xml') #Uploading Individual GEM p1#

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

#optimising the value#
solution=model1.optimize()
solution.objective_value
Vbiop5 = model1.slim_optimize() ## Biomass in individual GEM p5
Vbiop5 = round(Vbiop5,3)

cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

#optimising the value#
solution=model2.optimize()
solution.objective_value
Vbiop1 = model2.slim_optimize() ## Biomass in individual GEM p1
Vbiop1 = round(Vbiop1,3)

cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

#Definign species-specific metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"bsp"
        i.id=j
        i.compartment="c0bsp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ml"
        i.id=j
        i.compartment="c0ml"

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

#Definign species-specific reactions annotation

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

##Species-specific biomass reactions
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

##Defining community model
from cobra import Model, Reaction, Metabolite
model = Model('community_model4')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )

##Defining community biomass reaction
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0bsp"): -0.513,
                          model2.metabolites.get_by_id("cpd11416_c0ml"): -0.487,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining community objective function
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())
cobra.io.write_sbml_model(model,'Output_GCM/GCM_P1_P5.xml') ##Writing the communtiy model

##Running FBA
F = model.optimize()
flux_p1_p5=F.fluxes
flux_p1_p5.to_csv("Output_FBA/FBA_p1_p5.csv")

##Calculating PGSI##

Vbiocomp1p5 = model.reactions.Bio1.flux

Vbiocomp1 = model.reactions.bio1ml.flux ##Biomass of P1 in the community GEM
Vbiocomp1 = round(Vbiocomp1,3)

Vbiocomp5 = model.reactions.bio1bsp.flux  ##Biomass of P5 in the community GEM
Vbiocomp5 = round(Vbiocomp5, 3)

PGSIp5p1 = ((Vbiocomp1-Vbiop1)/Vbiop1)
PGSIp5p1 = round(PGSIp5p1,3)
PGSIp1p5 = ((Vbiocomp5-Vbiop5)/Vbiop5)
PGSIp1p5 = round(PGSIp1p5,3)

print('PGSI_P1-->P5:', PGSIp1p5)
print('PGSI_P5-->P1:', PGSIp5p1)
print('Avg PGSI: ', ((PGSIp1p5+PGSIp5p1)/2))

In [ ]:
##5Reconstruction of pairwise community model- GCM_P1_P6##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/ml_P1.xml') #Uploading Individual GEM p1#
model2 = cobra.io.read_sbml_model(r'Final_GEM/nc_P6.xml') #Uploading Individual GEM p6#

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

#optimising the model1#
solution=model1.optimize()
solution.objective_value
Vbiop1 = model1.reactions.bio1.flux ## Biomass in individual GEM p1
Vbiop1 = round(Vbiop1,3)

cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

#optimising the model2#
solution=model2.optimize()
solution.objective_value
Vbiop6 = model2.reactions.bio1.flux ## Biomass in individual GEM p6
Vbiop6 = round(Vbiop6,3)



cobra.flux_analysis.find_blocked_reactions(model2)

##Defining species-specific metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ml"
        i.id=j
        i.compartment="c0ml"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"nc"
        i.id=j
        i.compartment="c0nc"


##Defining species-specific reaction annotation

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

##Defining species-specific biomass reaction annotation

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)


count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ml"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

##Defining Community GEM

from cobra import Model, Reaction, Metabolite
model = Model('community_model5')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )

##Defining Community biomass based on the microbial growth rate
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ml"): -0.625,
                          model2.metabolites.get_by_id("cpd11416_c0nc"): -0.375,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining community biomass reaction
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())
cobra.io.write_sbml_model(model,'Output_GCM/GCM_P1_P6.xml') ##Writing the communtiy model

##Running FBA
F = model.optimize()
flux_p1_p6=F.fluxes
flux_p1_p6.to_csv("Output_FBA/FBA_p1_p6.csv")

##Calculating PGSI

Vbiocomp1p6 = model.reactions.Bio1.flux

Vbiocomp1 = model.reactions.bio1ml.flux ##Biomass of P1 in the community GEM
Vbiocomp1 = round(Vbiocomp1,3)

Vbiocomp6 = model.reactions.bio1nc.flux  ##Biomass of P6 in the community GEM
Vbiocomp6 = round(Vbiocomp6, 3)

PGSIp6p1 = ((Vbiocomp1-Vbiop1)/Vbiop1)
PGSIp6p1 = round(PGSIp6p1,3)
PGSIp1p6 = ((Vbiocomp6-Vbiop6)/Vbiop6)
PGSIp1p6 = round(PGSIp1p6,3)

print('PGSI_P1-->P6:', PGSIp1p6)
print('PGSI_P6-->P1:', PGSIp6p1)
print('Avg PGSI: ', ((PGSIp1p6+PGSIp6p1)/2))

In [ ]:
##7Reconstruction of pairwise community model- GCM_P2_P3##


model1 = cobra.io.read_sbml_model(r'Final_GEM/ksp_P2.xml') #Uploading Individual GEM p2#
model2 = cobra.io.read_sbml_model(r'Final_GEM/csp_P3.xml') #Uploading Individual GEM p3#

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

#optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop2 = model1.slim_optimize()## Biomass in individual GEM p2
Vbiop2 = model1.reactions.bio1.flux
Vbiop2 = round(Vbiop2,3)


cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))


#optimizing model2
solution=model2.optimize()
solution.objective_value
Vbiop3 = model2.slim_optimize()## Biomass in individual GEM p3
Vbiop3 = model2.reactions.bio1.flux
Vbiop3 = round(Vbiop3,3)

cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

##Defining species-specific metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ksp"
        i.id=j
        i.compartment="c0ksp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"csp"
        i.id=j
        i.compartment="c0csp"

##Defining species-specific reaction annotation
count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

##Defining species-specific biomass reaction
count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

##Defining Community GEM
from cobra import Model, Reaction, Metabolite
model = Model('community_model6')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )

##Defining Community Boiomass breaction based on microbial growth rate

reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ksp"): -0.63,
                          model2.metabolites.get_by_id("cpd11416_c0csp"): -0.37,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining Community Objective Function

model.objective = 'Bio1'

model.reactions.EX_cpd00009_e0.bounds=(-.14,.14)
print('Community model has been reconstructed')

print('Model objective Bio1: ', model.slim_optimize())
cobra.io.write_sbml_model(model,'Output_GCM/GCM_P2_P3.xml') ##Writing the communtiy model


## Running FBA
F = model.optimize()
flux_p2_p3=F.fluxes
flux_p2_p3.to_csv("Output_FBA/FBA_p2_p3.csv")

## calculating PGSI
Vbiocomp2p3 = model.reactions.Bio1.flux

Vbiocomp2 = model.reactions.bio1ksp.flux ##Biomass of P2 in the community GEM
Vbiocomp2 = round(Vbiocomp2,3)

Vbiocomp3 = model.reactions.bio1csp.flux  ##Biomass of P3 in the community GEM
Vbiocomp3 = round(Vbiocomp3, 3)

PGSIp3p2 = ((Vbiocomp2-Vbiop2)/Vbiop2)
PGSIp3p2 = round(PGSIp3p2,3)
PGSIp2p3 = ((Vbiocomp3-Vbiop3)/Vbiop3)
PGSIp2p3 = round(PGSIp2p3,3)

print('PGSI_P2-->P3:', PGSIp2p3)
print('PGSI_P3-->P2:', PGSIp3p2)
print('Avg PGSI: ', ((PGSIp2p3+PGSIp3p2)/2))


In [ ]:
##7Reconstruction of pairwise community model- GCM_P2_P4##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/ksp_P2.xml') ##Reading individual GEM P2
model2 = cobra.io.read_sbml_model(r'Final_GEM/osp_p4.xml') ##Reading individual GEM P4

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop2 = model1.slim_optimize() ## Biomass in individual GEM p2
Vbiop2 = model1.reactions.bio1.flux


cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

# optimizing model2
solution=model2.optimize()
solution.objective_value
Vbiop4 = model2.slim_optimize() ## Biomass in individual GEM p4
Vbiop4 = model2.reactions.bio1.flux
Vbiop4 = round(Vbiop4,3)


cobra.flux_analysis.find_blocked_reactions(model2)

##Definign Species-specific metabolite annotation

for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ksp"
        i.id=j
        i.compartment="c0ksp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"osp"
        i.id=j
        i.compartment="c0osp"
##Definign Species-specific reactions annotation
count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

##Definign Species-specific biomass reaction annotation
count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

##Defining Community GEM
from cobra import Model, Reaction, Metabolite
model = Model('community_model7')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
##Defining Community biomass reaction
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ksp"): -0.765,
                          model2.metabolites.get_by_id("cpd11416_c0osp"): -0.235,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining Community objective function
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P2_P4.xml') ##Writing the community model


##Running FBA
F = model.optimize()
flux_p2_p4=F.fluxes
flux_p2_p4.to_csv("Output_FBA/FBA_p2_p4.csv")


##Calculating PGSI
Vbiocomp2p4 = model.reactions.Bio1.flux

Vbiocomp2 = model.reactions.bio1ksp.flux ##Biomass of P2 in the community GEM
Vbiocomp2 = round(Vbiocomp2,3)

Vbiocomp4 = model.reactions.bio1osp.flux  ##Biomass of P4 in the community GEM
Vbiocomp4 = round(Vbiocomp4, 3)

PGSIp4p2 = ((Vbiocomp2-Vbiop2)/Vbiop2)
PGSIp4p2 = round(PGSIp4p2,3)
PGSIp2p4 = ((Vbiocomp4-Vbiop4)/Vbiop4)
PGSIp2p4 = round(PGSIp2p4,3)

print('PGSI_P2-->P4:', PGSIp2p4)
print('PGSI_P4-->P2:', PGSIp4p2)
print('Avg PGSI: ', ((PGSIp2p4+PGSIp4p2)/2))


In [ ]:
##8Reconstruction of pairwise community model- GCM_P2_P5##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/ksp_P2.xml') ##Uploading Individual GEM P2
model2 = cobra.io.read_sbml_model(r'Final_GEM/bsp_P5.xml') ##Uploading Individual GEM P5

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop2 = model1.slim_optimize() ## Biomass in individual GEM p2
Vbiop2 = model1.reactions.bio1.flux



cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

# optimizing model2
solution=model2.optimize()
solution.objective_value
Vbiop5 = model2.slim_optimize() ## Biomass in individual GEM p5
Vbiop5 = model2.reactions.bio1.flux


cobra.flux_analysis.find_blocked_reactions(model2)

##Deining species-specific metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ksp"
        i.id=j
        i.compartment="c0ksp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"bsp"
        i.id=j
        i.compartment="c0bsp"

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

##Deining species-specific reaction annotation

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

##Deining species-specific biomass reaction annotation

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)


##Definign community GEM
from cobra import Model, Reaction, Metabolite
model = Model('community_model8')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
##Defining Community biomass reactions Based on the microbial growth rate
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ksp"): -0.42,
                          model2.metabolites.get_by_id("cpd11416_c0bsp"): -0.58,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining community objective function
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P2_P5.xml') ##Writing the communtiy model


##Running FBA
F = model.optimize()
flux_p2_p5=F.fluxes
flux_p2_p5.to_csv("Output_FBA/FBA_p2_p5.csv")

##Calculating PGSI
Vbiocomp2p5 = model.reactions.Bio1.flux

Vbiocomp2 = model.reactions.bio1ksp.flux ##Biomass of P2 in the community GEM
Vbiocomp2 = round(Vbiocomp2,3)

Vbiocomp5 = model.reactions.bio1bsp.flux  ##Biomass of P5 in the community GEM
Vbiocomp5 = round(Vbiocomp5, 3)

PGSIp5p2 = ((Vbiocomp2-Vbiop2)/Vbiop2)
PGSIp5p2 = round(PGSIp5p2,3)
PGSIp2p5 = ((Vbiocomp5-Vbiop5)/Vbiop5)
PGSIp2p5 = round(PGSIp2p5,3)

print('PGSI_P2-->P5:', PGSIp2p5)
print('PGSI_P5-->P2:', PGSIp5p2)
print('Avg PGSI: ', ((PGSIp2p5+PGSIp5p2)/2))

In [ ]:
##9Reconstruction of pairwise community model- GCM_P2_P6##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/ksp_P2.xml') ##Uploading Individual GEM P2
model2 = cobra.io.read_sbml_model(r'Final_GEM/nc_P6.xml')  ##Uploading Individual GEM P6

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop2 = model1.slim_optimize() ## Biomass in individual GEM p2
Vbiop2 = round(Vbiop2,3)

cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

# optimizing model2
solution=model2.optimize()
solution.objective_value
Vbiop6 = model2.slim_optimize() ## Biomass in individual GEM p6
Vbiop6 = round(Vbiop6,3)

cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

##Defining Species-specific metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"ksp"
        i.id=j
        i.compartment="c0ksp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"nc"
        i.id=j
        i.compartment="c0nc"

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

##Defining Species-specific reaction annotation
count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

##Defining Species-specific biomass reaction annotation
count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"ksp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

##Definign Community GEM
from cobra import Model, Reaction, Metabolite
model = Model('community_model9')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )

##Defining Community biomass reaction based on microbial abundance
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0ksp"): -0.59,
                          model2.metabolites.get_by_id("cpd11416_c0nc"): -0.41,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining Community objective function
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P2_P6.xml') ##writing the communtiy model


##Running FBA
F = model.optimize()
flux_p2_p6=F.fluxes
flux_p2_p6.to_csv("Output_FBA/FBA_p2_p6.csv")

##Calculating PGSI
Vbiocomp2p6= model.reactions.Bio1.flux

Vbiocomp2 = model.reactions.bio1ksp.flux ##Biomass of P2 in the community GEM
Vbiocomp2 = round(Vbiocomp2,3)

Vbiocomp6 = model.reactions.bio1nc.flux  ##Biomass of P6 in the community GEM
Vbiocomp6 = round(Vbiocomp6, 3)

PGSIp6p2 = ((Vbiocomp2-Vbiop2)/Vbiop2)
PGSIp6p2 = round(PGSIp6p2,3)
PGSIp2p6 = ((Vbiocomp6-Vbiop6)/Vbiop6)
PGSIp2p6 = round(PGSIp2p6,3)

print('PGSI_P2-->P6:', PGSIp2p6)
print('PGSI_P6-->P2:', PGSIp6p2)
print('Avg PGSI: ', ((PGSIp2p6+PGSIp6p2)/2))

In [ ]:
##10Reconstruction of pairwise community model- GCM_P3_P4##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/csp_P3.xml') ##Uploading Individual GEM P3
model2 = cobra.io.read_sbml_model(r'Final_GEM/osp_P4.xml') ##Uploading Individual GEM P4


print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop3 = model1.slim_optimize() ## Biomass in individual GEM p3
Vbiop3 = model1.reactions.bio1.flux


cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

# optimizing model2
solution=model2.optimize()
solution.objective_value
Vbiop4 = model2.slim_optimize() ## Biomass in individual GEM p4
Vbiop4 = model2.reactions.bio1.flux


cobra.flux_analysis.find_blocked_reactions(model2)

##Defining Species-specific metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"csp"
        i.id=j
        i.compartment="c0csp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"osp"
        i.id=j
        i.compartment="c0osp"

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

##Defining species-specific reaction annotation
count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

##Defining community model
from cobra import Model, Reaction, Metabolite
model = Model('community_model10')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
##Defining community biomass reaction based on the microbial growth rate
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0csp"): -0.66,
                          model2.metabolites.get_by_id("cpd11416_c0osp"): -0.34,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))
model.reactions.EX_cpd00067_e0.bounds = (-.97,.97)

#Defining community objective function
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P3_P4.xml') ##Writing community model


##Running FBA
F = model.optimize()
flux_p3_p4=F.fluxes
flux_p3_p4.to_csv("Output_FBA/FBA_p3_p4.csv")

##Calculating PGSI
Vbiocomp3p4= model.reactions.Bio1.flux

Vbiocomp3 = model.reactions.bio1csp.flux ##Biomass of P2 in the community GEM
Vbiocomp3 = round(Vbiocomp3,3)

Vbiocomp4 = model.reactions.bio1osp.flux  ##Biomass of P4 in the community GEM
Vbiocomp4 = round(Vbiocomp4, 3)

PGSIp4p3 = ((Vbiocomp3-Vbiop3)/Vbiop3)
PGSIp4p3 = round(PGSIp4p3,3)
PGSIp3p4 = ((Vbiocomp4-Vbiop4)/Vbiop4)
PGSIp3p4 = round(PGSIp3p4,3)

print('PGSI_P3-->P4:', PGSIp3p4)
print('PGSI_P4-->P3:', PGSIp4p3)
print('Avg PGSI: ', ((PGSIp3p4+PGSIp4p3)/2))

In [ ]:
##11Reconstruction of pairwise community model- GCM_P3_P5##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/bsp_P5.xml') ##Uploading Individual GEM P5
model2 = cobra.io.read_sbml_model(r'Final_GEM/csp_P3.xml') ##Uploading Individual GEM P3

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop5 = model1.slim_optimize() ## Biomass in individual GEM p5
Vbiop5 = model1.reactions.bio1.flux


cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

#optimizing model2
solution=model2.optimize()
solution.objective_value

Vbiop3 = model2.slim_optimize() ## Biomass in individual GEM p3
Vbiop3 = model2.reactions.bio1.flux

cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

##definign Individual metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"bsp"
        i.id=j
        i.compartment="c0bsp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"csp"
        i.id=j
        i.compartment="c0csp"

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

#definign individual reaction annotation
count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

##Defining communtiy GEM
from cobra import Model, Reaction, Metabolite
model = Model('community_model1')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
##Definign community biomass reaction based on the microbial abundace
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0bsp"): -0.70,
                          model2.metabolites.get_by_id("cpd11416_c0csp"): -0.30,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

##Defining community objective function
model.objective = 'Bio1'

model.reactions.rxn05518_c0bsp.bounds = (-100,100)

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P3_P5.xml')##Writing communtiy model

##Running FAB
F = model.optimize()
flux_p3_p5=F.fluxes
flux_p3_p5.to_csv("Output_FBA/FBA_p3_p5.csv")


##Calculating PGSI
Vbiocomp3p5= model.reactions.Bio1.flux

Vbiocomp3 = model.reactions.bio1csp.flux ##Biomass of P3 in the community GEM
Vbiocomp3 = round(Vbiocomp3,3)

Vbiocomp5 = model.reactions.bio1bsp.flux  ##Biomass of P5 in the community GEM
Vbiocomp5 = round(Vbiocomp5, 3)

PGSIp5p3 = ((Vbiocomp3-Vbiop3)/Vbiop3)
PGSIp5p3 = round(PGSIp5p3,3)
PGSIp3p5 = ((Vbiocomp5-Vbiop5)/Vbiop5)
PGSIp3p5 = round(PGSIp3p5,3)

print('PGSI_P3-->P5:', PGSIp3p5)
print('PGSI_P5-->P3:', PGSIp5p3)
print('Avg PGSI: ', ((PGSIp3p5+PGSIp5p3)/2))

In [ ]:
##12Reconstruction of pairwise community model- GCM_P3_P6##

import cobra
import sys
import os

model1 = cobra.io.read_sbml_model(r'Final_GEM/nc_P6.xml') ##defining Individual GEM p6
model2 = cobra.io.read_sbml_model(r'Final_GEM/csp_P3.xml') ##defining Individual GEM p3

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

#optimising the value#
solution=model1.optimize()
solution.objective_value
Vbiop6 = model1.slim_optimize() ## Biomass in individual GEM p6
Vbiop6 = model1.reactions.bio1.flux


cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

#optimising the value#
solution=model2.optimize()
solution.objective_value
Vbiop3 = model2.slim_optimize() ## Biomass in individual GEM p3
Vbiop3 = model2.reactions.bio1.flux

cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

##Defining species-specific metabolite annotation
for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"csp"
        i.id=j
        i.compartment="c0csp"

for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"nc"
        i.id=j
        i.compartment="c0nc"

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)
##Defining species-specific biomass reaction
count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"csp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)
##Defining Community GEM
from cobra import Model, Reaction, Metabolite
model = Model('community_model2')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
##Defining community biomass reaction
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model2.metabolites.get_by_id("cpd11416_c0csp"): -0.43,
                          model1.metabolites.get_by_id("cpd11416_c0nc"): -0.57,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))
model.reactions.rxn05313_c0nc.bounds=(-.18,.18)

##Defining community objective
model.objective = 'Bio1'
print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())##writing the community GEM


cobra.io.write_sbml_model(model,'Output_GCM/GCM_P3_P6.xml')

##Running FBA

F = model.optimize()
flux_p3_p6=F.fluxes
flux_p3_p6.to_csv("Output_FBA/FBA_p3_p6.csv")

##Calculating PGSI
Vbiocomp3p6= model.reactions.Bio1.flux

Vbiocomp3 = model.reactions.bio1csp.flux ##Biomass of P3 in the community GEM
Vbiocomp3 = round(Vbiocomp3,3)

Vbiocomp6 = model.reactions.bio1nc.flux  ##Biomass of P6 in the community GEM
Vbiocomp6 = round(Vbiocomp6, 3)

PGSIp6p3 = ((Vbiocomp3-Vbiop3)/Vbiop3)
PGSIp6p3 = round(PGSIp6p3,3)
PGSIp3p6 = ((Vbiocomp6-Vbiop6)/Vbiop6)
PGSIp3p6 = round(PGSIp3p6,3)

print('PGSI_P3-->P6:', PGSIp3p6)
print('PGSI_P6-->P3:', PGSIp6p3)
print('Avg PGSI: ', ((PGSIp3p6+PGSIp6p3)/2))

In [ ]:
##13Reconstruction of pairwise community model- GCM_P4_P5##

import cobra
import sys
import os

model1 = cobra.io.read_sbml_model(r'Final_GEM/bsp_P5.xml')##Uploading individual GEM P5
model2 = cobra.io.read_sbml_model(r'Final_GEM/osp_P4.xml')##Uploading individual GEM P4

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop5 = model1.slim_optimize() ## Biomass in individual GEM p5


cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

# optimizing model2
solution=model2.optimize()
solution.objective_value
Vbiop4 = model2.slim_optimize() ## Biomass in individual GEM p4


cobra.flux_analysis.find_blocked_reactions(model2)
##Defining species-specific metabolite annottaion
for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"osp"
        i.id=j
        i.compartment="c0osp"

for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"bsp"
        i.id=j
        i.compartment="c0bsp"

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)
##Defining species-specific biomass reactions annotation
count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)
##Defining Community model
from cobra import Model, Reaction, Metabolite
model = Model('community_model3')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
##Defining community biomass reaction
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model2.metabolites.get_by_id("cpd11416_c0osp"): -0.28,
                          model1.metabolites.get_by_id("cpd11416_c0bsp"): -0.72,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))
##Defining community biomass reaction
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P4_P5.xml')##Writing Communtiy model


##Running FBA
F = model.optimize()
flux_p4_p5=F.fluxes
flux_p4_p5.to_csv("Output_FBA/FBA_p4_p5.csv")

##Calculating PGSI
Vbiocomp4p5= model.reactions.Bio1.flux

Vbiocomp4 = model.reactions.bio1osp.flux ##Biomass of P4 in the community GEM
Vbiocomp4 = round(Vbiocomp4,3)

Vbiocomp5 = model.reactions.bio1bsp.flux  ##Biomass of P5 in the community GEM
Vbiocomp5 = round(Vbiocomp5, 3)

PGSIp5p4 = ((Vbiocomp4-Vbiop4)/Vbiop4)
PGSIp5p4 = round(PGSIp5p4,3)
PGSIp4p5 = ((Vbiocomp5-Vbiop5)/Vbiop5)
PGSIp4p5 = round(PGSIp4p5,3)

print('PGSI_P4-->P5:', PGSIp4p5)
print('PGSI_P5-->P4:', PGSIp5p4)

In [ ]:
##14Reconstruction of pairwise community model- GCM_P4_P6##

import cobra

model1 = cobra.io.read_sbml_model(r'Final_GEM/nc_P6.xml') ##Uploading individual GEM P6
model2 = cobra.io.read_sbml_model(r'Final_GEM/osp_P4.xml') ##Uploading individual GEM P4

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop6 = model1.slim_optimize() ## Biomass in individual GEM p6
Vbiop6 = model1.reactions.bio1.flux



cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

#optimising the value#
solution=model2.optimize()
solution.objective_value
Vbiop4 = model2.slim_optimize() ## Biomass in individual GEM p4
Vbiop4 = model2.reactions.bio1.flux



cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##
#Defining microbe-specific metabolite annotation

for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"nc"
        i.id=j
        i.compartment="c0nc"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"osp"
        i.id=j
        i.compartment="c0osp"

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)
##Defining microbe-specific biomass reaction annotation
count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"osp"
        i.id=j
        count=count+1
print(count)
##Defining Communtiy model
from cobra import Model, Reaction, Metabolite
model = Model('community_model14')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
#Defining community biomass reaction
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0nc"): -0.62,
                          model2.metabolites.get_by_id("cpd11416_c0osp"): -0.38,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))
##Defining community model objective
model.objective = 'Bio1'

print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P4_P6.xml') ##writing the communtiy model

##Running FBA
F = model.optimize()
flux_p4_p6=F.fluxes
flux_p4_p6.to_csv("Output_FBA/FBA_p4_p6.csv")

##Calculating PGSI
Vbiocomp4p6= model.reactions.Bio1.flux

Vbiocomp4 = model.reactions.bio1osp.flux ##Biomass of P4 in the community GEM
Vbiocomp4 = round(Vbiocomp4,3)

Vbiocomp6 = model.reactions.bio1nc.flux  ##Biomass of P6 in the community GEM
Vbiocomp6 = round(Vbiocomp6, 3)

PGSIp6p4 = ((Vbiocomp4-Vbiop4)/Vbiop4)
PGSIp6p4 = round(PGSIp6p4,3)
PGSIp4p6 = ((Vbiocomp6-Vbiop6)/Vbiop6)
PGSIp4p6 = round(PGSIp4p6,3)

print('PGSI_P4-->P6:', PGSIp4p6)
print('PGSI_P6-->P4:', PGSIp6p4)
print('Avg PGSI: ', ((PGSIp4p6+PGSIp6p4)/2))

In [ ]:
##15Reconstruction of pairwise community model- GCM_P5_P6##

import cobra
import sys
import os

model1 = cobra.io.read_sbml_model(r'Final_GEM/bsp_P5.xml') ##Updating individua GEM P5
model2 = cobra.io.read_sbml_model(r'Final_GEM/nc_P6.xml') ##Updating individual GEM P6

print(len(model1.reactions))
print(len(model1.metabolites))
print(len(model1.genes))

# optimizing model1
solution=model1.optimize()
solution.objective_value
Vbiop5 = model1.slim_optimize() ## Biomass in individual GEM p6
Vbiop5 = model1.reactions.bio1.flux


cobra.flux_analysis.find_blocked_reactions(model1)

print(len(model2.reactions))
print(len(model2.metabolites))
print(len(model2.genes))

# optimizing model2
solution=model2.optimize()
solution.objective_value
Vbiop6 = model2.slim_optimize() ## Biomass in individual GEM p6
Vbiop6 = model2.reactions.bio1.flux

cobra.flux_analysis.find_blocked_reactions(model2)####use Jupyter notebook##

##Defining microbe-specific metabolite annotation
for i in model1.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"bsp"
        i.id=j
        i.compartment="c0bsp"

for i in model2.metabolites:
    if(str(i)[-2]=="c"):
        j=str(i)+"nc"
        i.id=j
        i.compartment="c0nc"

count=0
for i in model1.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)[0]=="r"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)
##Defining microbe-specific biomass reaction annotation
count=0
for i in model1.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="bio1"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)

count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"bsp"
        i.id=j
        count=count+1
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):continue
    if(str(i.id)[0]=="E" and str(i.id)[-2]=="c"):
        j=str(i.id)+"nc"
        i.id=j
        count=count+1
print(count)
##Defining Community model
from cobra import Model, Reaction, Metabolite
model = Model('community_model15')
count=0
for i in model1.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model1.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

count=0
for i in model2.reactions:
    if(str(i.id)=="EX_cpd11416_c0"):
        continue
    reaction=model2.reactions.get_by_id(str(i.id))
    reaction.reaction
    count=count+1
    model.add_reactions([reaction])
        
print(count)

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

from cobra import Metabolite
cpd11416_c0 = Metabolite(
    'cpd11416_c0',
    name='Complete_Biomass',
    compartment='c0'

    )

from cobra import Metabolite
cpd11416_b = Metabolite(
    'cpd11416_b',
    name='Complete_Biomass',
    compartment='e0'
    )
#Defining community biomass based on microbial growth rate
reaction = Reaction('Bio1')
reaction.name = ('Community_biomass')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({model1.metabolites.get_by_id("cpd11416_c0bsp"): -0.64,
                          model2.metabolites.get_by_id("cpd11416_c0nc"): -0.36,
                          cpd11416_c0: 1
                         })

        
reaction.reaction
model.add_reactions([reaction])
print(count)

reaction = Reaction('EX_cpd11416_c0')
reaction.name = ('EX_cpd11416_c0')
reaction.lower_bound = 0
reaction.upper_bound = 1000

reaction.add_metabolites({
                          model.metabolites.get_by_id("cpd11416_c0"): -1,
                          cpd11416_b: 1
                        })

        
reaction.reaction
model.add_reactions([reaction])

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))
##Definign community biomass reaction
model.objective = 'Bio1'
print('Community model has been reconstructed')
print('Model objective Bio1: ', model.slim_optimize())

cobra.io.write_sbml_model(model,'Output_GCM/GCM_P5_P6.xml')##Writing the community model

F = model.optimize()
flux_p5_p6=F.fluxes
flux_p5_p6.to_csv("Output_FBA/FBA_p5_p6.csv")

##Calculating PGSI
Vbiocomp5p6= model.reactions.Bio1.flux

Vbiocomp5 = model.reactions.bio1bsp.flux ##Biomass of P5 in the community GEM
Vbiocomp5 = round(Vbiocomp5,3)

Vbiocomp6 = model.reactions.bio1nc.flux  ##Biomass of P6 in the community GEM
Vbiocomp6 = round(Vbiocomp6, 3)

PGSIp6p5 = ((Vbiocomp5-Vbiop5)/Vbiop5)
PGSIp6p5 = round(PGSIp6p5,3)
PGSIp5p6 = ((Vbiocomp6-Vbiop6)/Vbiop6)
PGSIp5p6 = round(PGSIp5p6,3)

print('PGSI_P5-->P6:', PGSIp5p6)
print('PGSI_P6-->P5:', PGSIp6p5)
print('Avg PGSI: ', ((PGSIp5p6+PGSIp6p5)/2))